In [1]:
from ctra_kalman_filter import CTRAKalmanFilter
import numpy as np

kf = CTRAKalmanFilter()
measurement = np.array([100, 50, 1.57])  # Example: [x, y, theta]
mean, cov = kf.initiate(measurement)
print("Initial State:", mean)
print("Initial Covariance:", cov)


Initial State: [100.    50.     0.     1.57   0.     0.  ]
Initial Covariance: [[0.01 0.   0.   0.   0.   0.  ]
 [0.   0.01 0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.01 0.  ]
 [0.   0.   0.   0.   0.   0.01]]


In [2]:
dt = 1.0
mean, cov = kf.predict(mean, cov, dt)
print("Predicted State:", mean)
print("Predicted Covariance:", cov)


Predicted State: [100.    50.     0.     1.57   0.     0.  ]
Predicted Covariance: [[0.06 0.   0.   0.   0.   0.  ]
 [0.   0.06 0.   0.   0.   0.  ]
 [0.   0.   1.05 0.   0.   0.  ]
 [0.   0.   0.   0.3  0.   0.  ]
 [0.   0.   0.   0.   0.06 0.  ]
 [0.   0.   0.   0.   0.   0.06]]


In [29]:
from matching import fuse_motion_ctra

tracks = [...]  # Populate with mock track states
detections = [...]  # Populate with mock detection states
cost_matrix = np.zeros((len(tracks), len(detections)))  # Mock IoU costs
fused_cost = fuse_motion_ctra(kf, cost_matrix, tracks, detections)
print("Fused Cost Matrix:", fused_cost)


AttributeError: 'CTRAKalmanFilter' object has no attribute 'predict_state'

In [30]:
# Import necessary modules
import numpy as np
from ctra_kalman_filter import CTRAKalmanFilter

# Initialize the Kalman filter
kf = CTRAKalmanFilter()

# Test 1: Initialization
print("==== TEST 1: Initialization ====")
initial_measurement = np.array([100.0, 50.0, 1.57])  # [x, y, θ]
mean, covariance = kf.initiate(initial_measurement)
print("Initial State shape:", mean.shape)

print("Initialized Mean:")
print(mean)
print(mean.shape)
print("Initialized Covariance:")
print(covariance)
print(covariance.shape)

# Expected Output:
# mean = [100.0, 50.0, 0.0, 1.57, 0.0, 0.0]
# covariance = diagonal matrix with defined std squared

# Test 2: Prediction
print("\n==== TEST 2: Prediction ====")
predicted_mean, predicted_covariance = kf.predict(mean, covariance, dt=2.0)

print("Predicted Mean:")
print(predicted_mean)
print(predicted_mean.shape)
print("Predicted Covariance:")
print(predicted_covariance)
print(predicted_covariance.shape)

# Test 3: Update
measurement = np.array([102.0, 52.0])  # Pad with zeros
print('measurement shape:', measurement.shape)
updated_mean, updated_covariance = kf.update(predicted_mean, predicted_covariance, measurement)
print("Updated Mean:", updated_mean)
print("Updated Covariance:", updated_covariance)

# Test 4: Predict State (Alternative Method)
print("\n==== TEST 4: Predict State ====")
predicted_state = kf.predict_state(mean, dt=1.0)

print("Predicted State:")
print(predicted_state)

# Test 5: Multi Predict
print("\n==== TEST 5: Multi Predict ====")
means = np.array([[100.0, 50.0, 1.0, 1.57, 0.1, 0.01],
                  [200.0, 150.0, 2.0, 0.0, 0.2, 0.02]])
covariances = np.array([np.eye(6), np.eye(6)])
multi_predicted_means, multi_predicted_covariances = kf.multi_predict(means, covariances, dt=1.0)

print("Multi Predicted Means:")
print(multi_predicted_means)
print("Multi Predicted Covariances:")
print(multi_predicted_covariances)


==== TEST 1: Initialization ====
Initial State shape: (6,)
Initialized Mean:
[100.    50.     0.     1.57   0.     0.  ]
(6,)
Initialized Covariance:
[[0.01 0.   0.   0.   0.   0.  ]
 [0.   0.01 0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.01 0.  ]
 [0.   0.   0.   0.   0.   0.01]]
(6, 6)

==== TEST 2: Prediction ====
Predicted Mean:
[100.    50.     0.     1.57   0.     0.  ]
(6,)
Predicted Covariance:
[[0.06 0.   0.   0.   0.   0.  ]
 [0.   0.06 0.   0.   0.   0.  ]
 [0.   0.   1.05 0.   0.   0.  ]
 [0.   0.   0.   0.3  0.   0.  ]
 [0.   0.   0.   0.   0.06 0.  ]
 [0.   0.   0.   0.   0.   0.06]]
(6, 6)
measurement shape: (2,)


ValueError: operands could not be broadcast together with shapes (2,) (4,) 

In [31]:
# Import required modules
import numpy as np
import scipy.linalg
from ctra_kalman_filter import CTRAKalmanFilter

# Initialize the Kalman Filter
kf = CTRAKalmanFilter()

# Helper Function for Display
def display_test_result(test_name, result, expected=None):
    print(f"==== {test_name} ====")
    print("Result:")
    print(result)
    if expected is not None:
        print("Expected:")
        print(expected)
    print()

# Test 1: Initialization
print("\n==== TEST 1: Initialization ====")
measurement = np.array([100.0, 50.0, 1.0, 2.0])  # [x, y, a, h]
mean, covariance = kf.initiate(measurement)

display_test_result(
    "Initialization - Mean", mean, 
    expected=np.array([100.0, 50.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])
)
display_test_result(
    "Initialization - Covariance", covariance
)

# Test 2: Prediction
print("\n==== TEST 2: Prediction ====")
predicted_mean, predicted_covariance = kf.predict(mean, covariance)
display_test_result("Prediction - Mean", predicted_mean)
display_test_result("Prediction - Covariance", predicted_covariance)

# Test 3: Update
print("\n==== TEST 3: Update ====")
measurement_update = np.array([102.0, 52.0, 1.1, 2.1])  # [x, y, a, h]
updated_mean, updated_covariance = kf.update(predicted_mean, predicted_covariance, measurement_update)
display_test_result("Update - Mean", updated_mean)
display_test_result("Update - Covariance", updated_covariance)

# Test 4: Multi Prediction
print("\n==== TEST 4: Multi Prediction ====")
means = np.array([
    [100.0, 50.0, 1.0, 2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    [200.0, 150.0, 2.0, 3.0, 0.1, 1.0, 0.1, 0.1, 0.1, 0.1]
])
covariances = np.array([np.eye(10) for _ in range(len(means))])
multi_predicted_means, multi_predicted_covariances = kf.multi_predict(means, covariances)
display_test_result("Multi Prediction - Means", multi_predicted_means)
display_test_result("Multi Prediction - Covariances", multi_predicted_covariances)

# Test 5: Gating Distance
print("\n==== TEST 5: Gating Distance ====")
measurement_batch = np.array([
    [102.0, 52.0, 1.1, 2.1],
    [98.0, 48.0, 0.9, 1.9]
])
gating_distances = kf.gating_distance(mean, covariance, measurement_batch)
display_test_result("Gating Distances", gating_distances)



==== TEST 1: Initialization ====
==== Initialization - Mean ====
Result:
[100.  50.   0.   1.   0.   0.]
Expected:
[100.  50.   1.   2.   0.   0.   0.   0.   0.   0.]

==== Initialization - Covariance ====
Result:
[[0.01 0.   0.   0.   0.   0.  ]
 [0.   0.01 0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.25 0.   0.  ]
 [0.   0.   0.   0.   0.01 0.  ]
 [0.   0.   0.   0.   0.   0.01]]


==== TEST 2: Prediction ====
==== Prediction - Mean ====
Result:
[100.  50.   0.   1.   0.   0.]

==== Prediction - Covariance ====
Result:
[[0.06 0.   0.   0.   0.   0.  ]
 [0.   0.06 0.   0.   0.   0.  ]
 [0.   0.   1.05 0.   0.   0.  ]
 [0.   0.   0.   0.3  0.   0.  ]
 [0.   0.   0.   0.   0.06 0.  ]
 [0.   0.   0.   0.   0.   0.06]]


==== TEST 3: Update ====


ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 6)